# Importing Necessary Libraries

In [15]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [17]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [18]:
# newyork_data

In [19]:
neighborhoods_data = newyork_data['features'] # Features contain the relevant data

In [20]:
neighborhoods_data[0] # Checking the first item in the list

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

## CONVERTING THE LIST INTO A DATAFRAME

In [21]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [22]:
# We create a loop to fill in the data

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [23]:
#neighborhoods.head(20)

In [24]:
#checking the number of Borough and Neighbourhood
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [54]:
kl_df = neighborhoods
kl_df.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [26]:
kl_df.to_csv('newyork.csv')

#### Creating a Map of New York City with its Neighbourhood superimposed on it.

In [27]:
#get the coordinates of New York City, in New York
address = 'New York City, NY'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City, New York {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City, New York 40.7127281, -74.0060152.


In [28]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [55]:
# save the map as HTML file
map_newyork.save('map_kl.png')

# EXPLORING THE NEIGHBOURHOOD WITH FOURSQUARE

In [30]:
#DEFINING THE FOURSQUARE CREDENTIALS AND VERSION

CLIENT_ID = 'CCZMEC4N5W03WF3N5I2LK0YKPVTY3PBQTDDC5KJY4GRXLY04' # your Foursquare ID
CLIENT_SECRET = 'O0GWFSPPH5B33T14LAGHYQGMZ5L3JNK1LO5AGTM22EWK2VFO' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CCZMEC4N5W03WF3N5I2LK0YKPVTY3PBQTDDC5KJY4GRXLY04
CLIENT_SECRET:O0GWFSPPH5B33T14LAGHYQGMZ5L3JNK1LO5AGTM22EWK2VFO


## Checking for the top 100 venues within a radius of 2000

In [31]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CCZMEC4N5W03WF3N5I2LK0YKPVTY3PBQTDDC5KJY4GRXLY04&client_secret=O0GWFSPPH5B33T14LAGHYQGMZ5L3JNK1LO5AGTM22EWK2VFO&v=20180604&ll=40.7127281,-74.0060152&radius=2000&limit=100'

In [32]:
results = requests.get(url).json()
#results
print('Done')

Done


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues=[]
    for name, lat, neighborhood in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venues in venues for item in venues])
    
    nearby_venues.columns = ['Neighborhood', 
                         'Latitude', 
                         'Longitude', 
                         'VenueName', 
                         'VenueLatitude', 
                         'VenueLongitude', 
                         'VenueCategory']
    
    return(nearby_venues)

print('done')

done


In [34]:
venues_df = getNearbyVenues(names = kl_df['Neighborhood'], latitudes = kl_df['Latitude'],longitudes = kl_df['Longitude'])
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Wakefield,40.894705,-74.08174,Jersey Mike's Subs,40.894356,-74.076210,Sandwich Place
1,Wakefield,40.894705,-74.08174,Wawa,40.891580,-74.073059,Convenience Store
2,Wakefield,40.894705,-74.08174,Saddle Brook Diner,40.895038,-74.080262,Diner
3,Wakefield,40.894705,-74.08174,ReBar,40.892255,-74.073605,Lounge
4,Wakefield,40.894705,-74.08174,Bergen County Harley-Davidson,40.892579,-74.073350,Motorcycle Shop


### Checking for the number of Neighbourhood

In [35]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Allerton,100,100,100,100,100,100
Annadale,1,1,1,1,1,1
Arden Heights,1,1,1,1,1,1
Arlington,100,100,100,100,100,100
Arrochar,100,100,100,100,100,100
Arverne,100,100,100,100,100,100
Astoria,60,60,60,60,60,60
Astoria Heights,54,54,54,54,54,54
Auburndale,36,36,36,36,36,36


### Checking for number of Unique Category Curated

In [36]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 261 uniques categories.


In [37]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Sandwich Place', 'Convenience Store', 'Diner', 'Lounge',
       'Motorcycle Shop', 'Park', 'Gym / Fitness Center',
       'Mexican Restaurant', 'Burger Joint', 'Steakhouse', 'Pizza Place',
       'Gastropub', 'Martial Arts Dojo', 'Bagel Shop', 'Ice Cream Shop',
       'Bakery', 'Asian Restaurant', 'Breakfast Spot',
       'Italian Restaurant', 'Seafood Restaurant',
       'Latin American Restaurant', 'Video Game Store',
       'Fast Food Restaurant', 'Gourmet Shop', 'Japanese Restaurant',
       'Pharmacy', 'Donut Shop', 'Gym', 'Arcade', 'Cosmetics Shop',
       'Restaurant', 'Coffee Shop', 'Deli / Bodega', 'Dog Run',
       'Sporting Goods Shop', 'Supermarket', 'Bar', 'Sushi Restaurant',
       'Discount Store', 'Tanning Salon', 'Salon / Barbershop',
       'Fried Chicken Joint', 'American Restaurant',
       'Other Great Outdoors', 'Pool Hall', 'Hotel', 'Food & Drink Shop',
       'Wings Joint', 'Pet Store', 'Chinese Restaurant'], dtype=object)

In [38]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

# ANALYSE THE NEIGHBOURHOOD

In [39]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(24389, 262)


,Neighborhoods,Accessories Store,Advertising Agency,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Gym,Colombian Restaurant,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lighting Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Port,Print Shop,Pub,Racetrack,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Booth,Toll Plaza,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped.head()

(299, 262)


,Neighborhoods,Accessories Store,Advertising Agency,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Gym,Colombian Restaurant,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lighting Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Port,Print Shop,Pub,Racetrack,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Booth,Toll Plaza,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allerton,0.0,0.0,0.0,0.01,0.06,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.01,0.03,0.04,0.03,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.01,0.01,0.00,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.01,0.0,0.01,0.0,0.02,0.0,0.0,0.0,0.0,0.00,0.04,0.02,0.0,0.0,0.00,0.0,0

In [41]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

15

Let's Create a new DataFrame for Shopping Mall data only

In [42]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

kl_mall.head()

,Neighborhoods,Shopping Mall
0,Allerton,0.0
1,Annadale,0.0
2,Arden Heights,0.0
3,Arlington,0.0
4,Arrochar,0.0


# CLUSTER THE NEIGHBOURHOOD

Run k-means to cluster the neighborhoods in New York City into 3 clusters.

In [43]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [44]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [45]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Allerton,0.0,0
1,Annadale,0.0,0
2,Arden Heights,0.0,0
3,Arlington,0.0,0
4,Arrochar,0.0,0


In [46]:
# merge new york city grouped with new york data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.tail() # check the last columns!

(303, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
294,Woodhaven,0.0,0,Queens,40.689887,-73.858110
295,Woodlawn,0.0,0,Bronx,40.898273,-73.867315
296,Woodrow,0.0,0,Staten Island,40.541968,-74.205246
297,Woodside,0.0,0,Queens,40.746349,-73.901842
298,Yorkville,0.0,0,Manhattan,40.775930,-73.947118


In [47]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(303, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.000000,0,Bronx,40.865788,-73.859319
201,Olinville,0.000000,0,Bronx,40.871371,-73.863324
200,Old Town,0.000000,0,Staten Island,40.596329,-74.087511
199,Ocean Parkway,0.000000,0,Brooklyn,40.613060,-73.968367
198,Ocean Hill,0.000000,0,Brooklyn,40.678403,-73.913068
197,Oakwood,0.000000,0,Staten Island,40.558462,-74.121566
196,Oakland Gardens,0.000000,0,Queens,40.745619,-73.754950
202,Ozone Park,0.000000,0,Queens,40.680708,-73.843203
195,Norwood,0.000000,0,Bronx,40.877224,-73.879391
192,North Corona,0.000000,0,Queens,40.754071,-73.857518


## VISUALISING THE RESULT

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Examining The Clusters

Cluster 0

In [50]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.0,0,Bronx,40.865788,-73.859319
201,Olinville,0.0,0,Bronx,40.871371,-73.863324
200,Old Town,0.0,0,Staten Island,40.596329,-74.087511
199,Ocean Parkway,0.0,0,Brooklyn,40.613060,-73.968367
198,Ocean Hill,0.0,0,Brooklyn,40.678403,-73.913068
197,Oakwood,0.0,0,Staten Island,40.558462,-74.121566
196,Oakland Gardens,0.0,0,Queens,40.745619,-73.754950
202,Ozone Park,0.0,0,Queens,40.680708,-73.843203
195,Norwood,0.0,0,Bronx,40.877224,-73.879391
192,North Corona,0.0,0,Queens,40.754071,-73.857518


Cluster 1

In [51]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
143,Kew Gardens,0.010309,1,Queens,40.705179,-73.829819
110,Glendale,0.010000,1,Queens,40.702762,-73.870742
139,Jamaica Center,0.010101,1,Queens,40.704657,-73.796902
106,Fulton Ferry,0.010000,1,Brooklyn,40.703281,-73.995508
290,Williamsburg,0.010000,1,Brooklyn,40.707144,-73.958115
279,Vinegar Hill,0.010000,1,Brooklyn,40.703321,-73.981116
46,Central Harlem,0.010526,1,Manhattan,40.815976,-73.943211
156,Longwood,0.011236,1,Bronx,40.815099,-73.895788
93,Financial District,0.010000,1,Manhattan,40.707107,-74.010665
193,North Riverdale,0.010000,1,Bronx,40.908543,-73.904531


Cluster 2

In [56]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
136,Hunts Point,0.016949,2,Bronx,40.80973,-73.883315
177,Morningside Heights,0.014706,2,Manhattan,40.80800,-73.963896


### CONCLUSION

The majority of the shopping Malls are concentrated around Manhattan, Bronx Whitestone Bridge, and Throg Necks Bridge. Cluster one has the highest number of shopping malls and a small number in cluster two. 

On the other hand, cluster zero which comprises of the majority of the neighborhood in New York has no shopping mall. This presents a great opportunity for property developers to open shopping malls in the neighborhood of cluster zero. 

Property developers should avoid neighborhood in cluster one, because of the competition the shopping malls in the neighborhood can generate.